In [ ]:
---
layout: post
title: Full Stack Feature Blog
categories: [Tri 2 Documentation]
comments: true
type: issues
---

## Create Performance Task

### Program Code

```python
class HobbyResource(Resource):
    def get(self):
        category = request.args.get('category', 'general')
        hobbies = Hobby.query.filter_by(category=category).all()
        if hobbies:
            return jsonify({"category": category, "hobbies": [hobby.name for hobby in hobbies]})
        else:
            return jsonify({"message": "Category not found"}), 404
    
    def post(self):
        data = request.get_json()
        if not data or not data.get('name') or not data.get('category'):
            return jsonify({"message": "Hobby name and category are required"}), 400
        
        hobby = Hobby(name=data['name'], category=data['category'])
        if hobby.create():
            return jsonify({"message": "Hobby created", "hobby": hobby.name, "category": hobby.category})
        else:
            return jsonify({"message": "Error creating hobby"}), 500

    def put(self):
        data = request.get_json()
        if not data or not data.get('name') or not data.get('category') or not data.get('old_name'):
            return jsonify({"message": "Hobby name, old name, and category are required to update"}), 400
        
        hobby = Hobby.query.filter_by(name=data['old_name'], category=data['category']).first()
        if not hobby:
            return jsonify({"message": "Hobby not found"}), 404
        
        hobby.name = data['name']
        if hobby.update():
            return jsonify({"message": "Hobby updated", "old_name": data['old_name'], "new_name": hobby.name})
        else:
            return jsonify({"message": "Error updating hobby"}), 500

    def delete(self):
        data = request.get_json()
        if not data or not data.get('name') or not data.get('category'):
            return jsonify({"message": "Hobby name and category are required to delete"}), 400
        
        hobby = Hobby.query.filter_by(name=data['name'], category=data['category']).first()
        if not hobby:
            return jsonify({"message": "Hobby not found"}), 404
        
        if hobby.delete():
            return jsonify({"message": "Hobby deleted", "name": hobby.name})
        else:
            return jsonify({"message": "Error deleting hobby"}), 500

api.add_resource(HobbyResource, '/hobby')
```

### PPR

- **How is data stored in a list?**

```python
hobbies = Hobby.query.filter_by(category=category).all()
```
**FRQ Notes**
- `Hobby.query` queries the SQLite database table for Hobbies
- `filter_by(category=category)` filters the results of the query based on where the `category` column matches the `category` table
- `.all()` executes the query and returns hobbies as a list of objects

- **How is a list being processed?**

```python
return jsonify({"category": category, "hobbies": [hobby.name for hobby in hobbies]})
```
**FRQ Notes**
- `[hobby.name for hobby in hobbies]` iterates through each object of the Hobby list, extracts the name of each attribute of the object, and returns a new list with just the names
- This list of names is inserted into the dictionary and `jsonify()` converts it into a JSON response

- **Procedure with parameter, selection, sequencing, iteration**

```python
def get(self):
        category = request.args.get('category', 'general')
        hobbies = Hobby.query.filter_by(category=category).all()
        if hobbies:
            return jsonify({"category": category, "hobbies": [hobby.name for hobby in hobbies]})
        else:
            return jsonify({"message": "Category not found"}), 404
```
**FRQ Notes**

Function: `get(self)`
- Purpose: Handles GET requests, retrieves hobbies based on category.

Steps:
- Get Category: `category = request.args.get('category', 'general')`
- Query Database: `hobbies = Hobby.query.filter_by(category=category).all()`
- Check if Hobbies Exist: `if hobbies:`
  - If yes, return names of hobbies in JSON: `return jsonify({"category": category, "hobbies": [hobby.name for hobby in hobbies]})`
  - If no, return error: `else: return jsonify({"message": "Category not found"}), 404`

Key Concepts:
- Parameter: Get `category` from query.
- Sequencing: Extract, query, check, return.
- Selection: Decision on whether hobbies exist.
- Iteration: List comprehension to extract names.

- **Call to procedure**

```js
    async function fetchHobbies() {
        try {
            const category = document.getElementById('category').value;
            const response = await fetch(`${pythonURI}/api/hobby?category=${category}`, {
                ...fetchOptions,
                method: 'GET'
            });

            if (!response.ok) {
                throw new Error('Failed to fetch hobbies: ' + response.statusText);
            }
```
**FRQ Notes**
- The JavaScript function sends a GET request to the server at `/api/hobby?category={category}`.
- The Flask `get` function on the server processes this request, extracts the category, queries the database, and returns a JSON response.
- The JavaScript function waits for the response and processes the result.

### Next Steps
- Implement a program that allows only admins to delete hobbies from the hobby log
- Make the hobbies log more dynamic by allowing users to click on a hobby and favorite it for future reference